## 데이터 전처리

In [49]:
# 1. 결측치 제거 및 대체
# 2. 이상치 확인(피쳐 엔지니어링 후)
# 3. 타겟 라벨링

In [40]:
# L.RD, L.Vac, R.RD, R.Vac : 타겟
# 그 외 : 피쳐 

In [ ]:
# 한글, 음수 시각화 설정

#시각화 패키지
import matplotlib.pyplot as plt

#시각화에서 한글을 사용하기 위한 설정
import platform
from matplotlib import font_manager, rc

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
    
#윈도우의 경우
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
    
#시각화에서 음수를 표현하기 위한 설정
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False 

import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN

## 2. 이상치 확인

- feature extraction 완료 후 데이터프레임

In [6]:
def detect_outliers(df) :
  
  outlier_indices = {}
  
  for col in feat_extraction_df.select_dtypes(include= np.number).columns :
    q1 = feat_extraction_df[col].quantile(0.25)
    q3 = feat_extraction_df[col].quantile(0.75)
    IQR = q3 - q1
    
    lower_bound = q1 - 1.5 * IQR
    upper_bound = q3 + 1.5 * IQR
    
    outlier_indices[col] = feat_extraction_df[(feat_extraction_df[col] < lower_bound) | (feat_extraction_df[col] > upper_bound)].index.tolist()
    
  return outlier_indices
  
outliers = detect_outliers(feat_extraction_df)


In [10]:
df = pd.read_csv('../data/process_extraction_mean_df.csv', index_col= 0)
df.info()
# df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 8145 entries, 2016-01-01 07 to 2016-12-31 23
Columns: 208 entries, LIFT.OUT.ROL.MTR.M.SPD to R.Vac
dtypes: float64(202), int64(6)
memory usage: 13.0+ MB


In [16]:
feature_df = df.iloc[:, :-14] # 투입량 포함
target_df = df[['L.RD', 'L.Vac', 'R.RD', 'R.Vac']]

In [12]:
feature_df.info() # 827
target_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8145 entries, 2016-01-01 07 to 2016-12-31 23
Columns: 194 entries, LIFT.OUT.ROL.MTR.M.SPD to STD.GLS.R
dtypes: float64(194)
memory usage: 12.1+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 8145 entries, 2016-01-01 07 to 2016-12-31 23
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   L.RD    8145 non-null   float64
 1   L.Vac   8145 non-null   float64
 2   R.RD    8145 non-null   float64
 3   R.Vac   8145 non-null   float64
dtypes: float64(4)
memory usage: 318.2+ KB


> ### 결측치 제거 및 대체

In [18]:
# 결측치 확인
print("결측치 (Feature):")
print(feature_df.isnull().sum()[feature_df.isnull().sum() > 0])
print("\n결측치 (Target):")
print(target_df.isnull().sum()[target_df.isnull().sum() > 0])

결측치 (Feature):
Ribbon.Speed                     1
UPPER.RET.Z.TMP               8145
X.PV                             1
DCS.Offset.RPM.M.PV              1
X.LEHR.SO2.ANALYZER              1
X.LEHR.O2.ANALYZER               1
CASHING.COLD.AIR.TMP             1
CASHING.HOT.AIR.TMP              1
DB.N2.SCREEN.FLOW.RIGHT.PV       1
EXIT.LIP.PLATE.LEFT              1
EXIT.LIP.PLATE.RIGHT             1
N2.TO.EA1313A.PV                 1
N2.TO.HOOD.DOWN.STREAM           1
N2.TO.HOOD.UP.STREAM             1
HOOD.N2.TOP.HEATER.L             1
HOOD.N2.HEATER.R                 1
DROSS.BOX.N2.BTM.HT.L            1
DROSS.BOX.N2.BTM.HT.R            1
DROSSBOX.N2.SCN.L                1
DROSSBOX.N2.SCN.R                1
DB.HEAT.TOP.TMP.CT               1
DB.HEAT.BOTTOM                   1
ATM.FLOW.SECT.N2.PV              1
SEC11.N2.PV                      1
SEC11.N2.H2.PV                   1
D.C.L.LR                         1
D.C.L.UL                         1
D.C.R.UL                         1
S.AB.

- 열 제거

In [32]:
feature_df['UPPER.RET.Z.TMP'] # 원본 263열

origin_df = pd.read_csv('../data/factory_glass_2016.csv', index_col= 0, encoding= 'cp949')
origin_df.head()
origin_df.iloc[:, 262] # 마찬가지로 결측열

dat...date.name.
2016-01-01 07   NaN
2016-01-01 08   NaN
2016-01-01 09   NaN
2016-01-01 10   NaN
2016-01-01 11   NaN
                 ..
2016-12-31 19   NaN
2016-12-31 20   NaN
2016-12-31 21   NaN
2016-12-31 22   NaN
2016-12-31 23   NaN
Name: UPPER.RET..Z218.TMP.1TIC42601.PV, Length: 8145, dtype: float64

In [33]:
feature_df.drop(labels= 'UPPER.RET.Z.TMP', axis= 1, inplace= True)
feature_df.info() # 311 > 310

<class 'pandas.core.frame.DataFrame'>
Index: 8145 entries, 2016-01-01 07 to 2016-12-31 23
Columns: 193 entries, LIFT.OUT.ROL.MTR.M.SPD to STD.GLS.R
dtypes: float64(193)
memory usage: 12.3+ MB


In [34]:
df = pd.concat([feature_df, target_df], axis= 1)
df.head()

,LIFT.OUT.ROL.MTR.M.SPD,LEHR.DRV.MTR.M.SPD,DRV.MTR.M.SPD,X,DCS.Offset.RPM.M,DAMP.POSITION.BAY.CTRL,DB.N2.SCREEN.FLOW.LEFT,DB.N2.SCREEN.FLOW.RIGHT,N2.TO.EA1313A,ROOF.HEATING.Zone,...,STD.U.L,STD.U.R,STD.L.L,STD.L.R,STD.GLS.L,STD.GLS.R,L.RD,L.Vac,R.RD,R.Vac
dat...date.name.,,,,,,,,,,,,,,,,,,,,,
2016-01-01 07,-1.742443,-1.699444,-1.683797,-0.755452,-1.947792,-1.251006,-1.514135,-1.529208,-0.218748,0.040366,...,0.523950,1.951349,2.248106,2.767676,3.012706,3.069007,0.025,0.063,0.108,0.041
2016-01-01 08,-1.744941,-1.699677,-1.685770,-0.755452,-1.947792,-1.251006,-1.514135,-1.529208,-0.218748,0.028982,...,0.566521,2.010125,2.364418,2.842630,3.022998,3.069274,0.000,0.022,0.133,0.000
2016-01-01 09,-1.744941,-1.699677,-1.685770,-0.755452,-1.947792,-1.251006,-1.514135,-1.529208,-0.218748,0.065345,...,0.545936,2.002820,2.397034,2.817728,2.936967,2.969952,0.016,0.063,0.000,0.000
2016-01-01 10,-1.744941,-1.699677,-1.685770,-0.755452,-1.947792,-1.251006,-1.514135,-1.529208,-0.218748,0.084378,...,0.549254,2.047995,2.399854,2.810619,2.876398,2.914596,0.000,0.103,0.000,0.000
2016-01-01 11,-1.744941,-1.699677,-1.685770,-0.755452,-1.947792,-1.251006,-1.514135,-1.529208,-0.218748,0.024016,...,0.610913,2.101178,2.379341,2.797094,2.858177,2.873929,0.000,0.000,0.000,0.000


- 행 제거

In [35]:
missing_rows = df[df.isnull().any(axis=1)]
print(f"결측치가 있는 행 수: {missing_rows.shape[0]}")

missing_idx = df.index[df.isnull().any(axis=1)]
print(missing_idx)
print()


missing_columns = df.columns[df.isnull().sum() > 0]
print(missing_columns)

df[missing_columns].loc[missing_idx]

결측치가 있는 행 수: 3
Index(['2016-06-24 04', '2016-10-13 09', '2016-12-27 17'], dtype='object', name='dat...date.name.')

Index(['Ribbon.Speed', 'X.PV', 'DCS.Offset.RPM.M.PV', 'X.LEHR.SO2.ANALYZER',
       'X.LEHR.O2.ANALYZER', 'CASHING.COLD.AIR.TMP', 'CASHING.HOT.AIR.TMP',
       'DB.N2.SCREEN.FLOW.RIGHT.PV', 'EXIT.LIP.PLATE.LEFT',
       'EXIT.LIP.PLATE.RIGHT', 'N2.TO.EA1313A.PV', 'N2.TO.HOOD.DOWN.STREAM',
       'N2.TO.HOOD.UP.STREAM', 'HOOD.N2.TOP.HEATER.L', 'HOOD.N2.HEATER.R',
       'DROSS.BOX.N2.BTM.HT.L', 'DROSS.BOX.N2.BTM.HT.R', 'DROSSBOX.N2.SCN.L',
       'DROSSBOX.N2.SCN.R', 'DB.HEAT.TOP.TMP.CT', 'DB.HEAT.BOTTOM',
       'ATM.FLOW.SECT.N2.PV', 'SEC11.N2.PV', 'SEC11.N2.H2.PV', 'D.C.L.LR',
       'D.C.L.UL', 'D.C.R.UL', 'S.AB.C.L.L', 'S.AB.C.L.R', 'S.C.C.L.L',
       'S.C.C.L.R'],
      dtype='object')


,Ribbon.Speed,X.PV,DCS.Offset.RPM.M.PV,X.LEHR.SO2.ANALYZER,X.LEHR.O2.ANALYZER,CASHING.COLD.AIR.TMP,CASHING.HOT.AIR.TMP,DB.N2.SCREEN.FLOW.RIGHT.PV,EXIT.LIP.PLATE.LEFT,EXIT.LIP.PLATE.RIGHT,...,ATM.FLOW.SECT.N2.PV,SEC11.N2.PV,SEC11.N2.H2.PV,D.C.L.LR,D.C.L.UL,D.C.R.UL,S.AB.C.L.L,S.AB.C.L.R,S.C.C.L.L,S.C.C.L.R
dat...date.name.,,,,,,,,,,,,,,,,,,,,,
2016-06-24 04,NaN,0.192755,0.065878,0.058545,-0.055645,0.859771,1.079344,0.263190,0.432926,0.436657,...,-0.337687,-0.466767,0.293627,-0.257381,-0.317551,-0.185653,-1.455076,-1.492665,0.071116,0.152644
2016-10-13 09,-0.835517,0.222480,0.065867,0.672414,0.250621,-0.381985,-0.715413,NaN,0.785044,0.443715,...,0.675100,-0.493571,0.315467,0.379425,-0.538530,-0.652667,-0.267265,-0.038846,-0.096260,-0.248660
2016-12-27 17,-0.832082,NaN,NaN,NaN,NaN,NaN,NaN,1.346741,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df.drop(labels= missing_idx, axis= 0, inplace= True)

In [37]:
missing_rows = df[df.isnull().any(axis=1)]
print(f"결측치가 있는 행 수: {missing_rows.shape[0]}")

결측치가 있는 행 수: 0


In [38]:
df.to_csv('../data/prp_factory_glass_2016.csv')

In [13]:
# 불량에 영향을 미치는 공정을 찾는 분석으로 이상치가 불량 여부에 영향을 미칠 수 있으므로
# 따로 이상치 처리를 하지 않음